## DPO代码实现


DPO 损失函数形式：
    $$L_{\text{DPO}}(\theta) = - \mathbb{E}_{(x, y^+, y^-)} \left[
    \log \sigma\left(
    \beta \cdot \left(
    \log\frac{\pi_\theta(y^+|x)}{\pi_\text{ref}(y^+|x)} - \log \frac{\pi_{\theta}(y^-|x)}{\pi_{\text{ref}}(y^-|x)} 
    \right)
    \right)
    \right]$$


其中：

- $\pi_\theta$：正在训练的语言模型；
- $\pi_{\text{ref}}$：参考模型（通常是SFT模型）；
- $\beta$：温度系数，控制对偏好的敏感程度。

In [1]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from icecream import ic

# ========== 初始化 GPT2 模型 ==========
device = "cuda" if torch.cuda.is_available() else "cpu"
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
ref_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
ref_model.eval()  # 参考模型不更新

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 512 

# ========== 构造假的DPO输入 ==========
# 每个prompt有 chosen / rejected 各一条
prompts = [
    "What is the capital of France?",
    "Explain quantum entanglement simply.",
    "Write a short poem about rain."
]
chosens = [
    "The capital of France is Paris.",
    "Quantum entanglement means two particles stay linked no matter the distance.",
    "Rain falls softly, painting the world anew."
]
rejecteds = [
    "France is a country.",
    "Quantum is physics.",
    "Rain is wet."
]

# 拼接 (chosen, rejected)
pairs = []
for p, c, r in zip(prompts, chosens, rejecteds):
    pairs.append(p + " " + c)
    pairs.append(p + " " + r)
    ic(len(pairs[-1]))

encodings = tokenizer(
    pairs, padding=True, truncation=True, return_tensors="pt"
).to(device)

input_ids = encodings.input_ids
attention_mask = encodings.attention_mask

ic(input_ids.shape)

# ========= 定义DPO核心逻辑 =========
def dpo_loss(policy_logps, ref_logps, beta=0.1):
    chosen = policy_logps[0::2]
    rejected = policy_logps[1::2]
    ref_chosen = ref_logps[0::2]
    ref_rejected = ref_logps[1::2]
    logits = (chosen - rejected) - (ref_chosen - ref_rejected)
    return -F.logsigmoid(beta * logits).mean()

def get_logps(model, input_ids, attention_mask):
    """返回每个样本的 log π(y|x),即交叉熵"""
    # with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits[:, :-1, :]
    labels = input_ids[:, 1:]
    log_probs = F.log_softmax(logits, dim=-1)
    # ic(log_probs.shape)     #log_probs.shape: torch.Size([6, 21, 50257])
    ic(labels.unsqueeze(2).shape)
    token_logps = torch.gather(log_probs, 2, labels.unsqueeze(2)).squeeze(2)
    # ic(token_logps.shape)       #token_logps.shape: torch.Size([6, 21])
    # ic(attention_mask.shape)        #attention_mask.shape: torch.Size([6, 22])
    seq_logps = (token_logps * attention_mask[:, 1:]).sum(dim=1)
    ic(seq_logps.shape)
    return -seq_logps

# ========= 优化器与训练 =========
opt = torch.optim.AdamW(model.parameters(), lr=1e-6)
beta = 0.1

for step in range(3):
    # 计算策略与参考模型 log π(y|x)
    model.train()
    policy_logps=get_logps(model,input_ids,attention_mask)

    with torch.no_grad():
        ref_logps = get_logps(ref_model, input_ids, attention_mask)

    # DPO loss
    loss = dpo_loss(policy_logps, ref_logps, beta)
    opt.zero_grad()
    loss.backward()
    opt.step()

    print(f"Step {step}: DPO loss = {loss.item():.4f}")


/opt/miniconda3/envs/trl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ic| len(pairs[-1]): 51
ic| len(pairs[-1]): 56
ic| len(pairs[-1]): 43
ic| input_ids.shape: torch.Size([6, 22])
ic| labels.unsqueeze(2).shape: torch.Size([6, 21, 1])
ic| seq_logps.shape: torch.Size([6])
ic| labels.unsqueeze(2).shape: torch.Size([6, 21, 1])
ic| seq_logps.shape: torch.Size([6])


Step 0: DPO loss = 0.6722


ic| labels.unsqueeze(2).shape: torch.Size([6, 21, 1])
ic| seq_logps.shape: torch.Size([6])
ic| labels.unsqueeze(2).shape: torch.Size([6, 21, 1])
ic| seq_logps.shape: torch.Size([6])


Step 1: DPO loss = 0.4389


ic| labels.unsqueeze(2).shape: torch.Size([6, 21, 1])
ic| seq_logps.shape: torch.Size([6])
ic| labels.unsqueeze(2).shape: torch.Size([6, 21, 1])
ic| seq_logps.shape: torch.Size([6])


Step 2: DPO loss = 0.7623


## 交叉熵的计算细节

1. **`logits[:, :-1, :]` 和 `labels = input_ids[:, 1:]` 的错位切片操作**

    是因为模型在每个位置上都使用前面所有 token 作为条件，去预测下一个 token。


    假设原始输入序列 input_ids 为 [x₁, x₂, x₃, x₄]（长度为 4），切片后：

    - 模型输出截断（logits [:, :-1, :]）：取所有样本、除最后一个 token 外的所有位置，得到 [x₁, x₂, x₃]（长度为 3）。这部分是模型的 “预测输入上下文”，用于生成对下一个 token 的预测。

    - 目标标签偏移（labels = input_ids [:, 1:]）：取所有样本、从第二个 token 开始的所有位置，得到 [x₂, x₃, x₄]（长度为 3）。这部分是模型的 “真实目标”，每个位置的标签对应前序上下文要预测的 token。



2. **负号 `-seq_logps`**

    在语言模型训练中，我们最小化的交叉熵损失：
    $$\mathcal{L}_{CE} = - \sum_t \log P_\theta(x_t | x_{<t})$$
    即交叉熵（Cross-Entropy）就是负的 token 级对数似然（Negative Log-Likelihood, NLL）。
    故而，本质上等价于**最大化每个 token 的对数似然**（log-likelihood）。
    所以在代码中返回 `-seq_logps`，


代码实现：
```python
def get_logps(model, input_ids, attention_mask):
    """返回每个样本的 log π(y|x),即交叉熵"""
    # with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits[:, :-1, :]
    labels = input_ids[:, 1:]
    log_probs = F.log_softmax(logits, dim=-1)
    # ic(log_probs.shape)     #log_probs.shape: torch.Size([6, 21, 50257])
    token_logps = torch.gather(log_probs, 2, labels.unsqueeze(2)).squeeze(2)
    # ic(token_logps.shape)       #token_logps.shape: torch.Size([6, 21])
    # ic(attention_mask.shape)        #attention_mask.shape: torch.Size([6, 22])
    seq_logps = (token_logps * attention_mask[:, 1:]).sum(dim=1)
    return -seq_logps
```
